# **Download the dataset**

In [ ]:
# download the dataset
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2024-01-25 18:25:24--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  19.9MB/s    in 4.6s    

2024-01-25 18:25:28 (17.3 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [ ]:
# extract the data
!tar -zxvf aclImdb_v1.tar.gz

Streaming output truncated to the last 5000 lines.
aclImdb/train/unsup/44983_0.txt
aclImdb/train/unsup/44982_0.txt
aclImdb/train/unsup/44981_0.txt
aclImdb/train/unsup/44980_0.txt
aclImdb/train/unsup/44979_0.txt
aclImdb/train/unsup/44978_0.txt
aclImdb/train/unsup/44977_0.txt
aclImdb/train/unsup/44976_0.txt
aclImdb/train/unsup/44975_0.txt
aclImdb/train/unsup/44974_0.txt
aclImdb/train/unsup/44973_0.txt
aclImdb/train/unsup/44972_0.txt
aclImdb/train/unsup/44971_0.txt
aclImdb/train/unsup/44970_0.txt
aclImdb/train/unsup/44969_0.txt
aclImdb/train/unsup/44968_0.txt
aclImdb/train/unsup/44967_0.txt
aclImdb/train/unsup/44966_0.txt
aclImdb/train/unsup/44965_0.txt
aclImdb/train/unsup/44964_0.txt
aclImdb/train/unsup/44963_0.txt
aclImdb/train/unsup/44962_0.txt
aclImdb/train/unsup/44961_0.txt
aclImdb/train/unsup/44960_0.txt
aclImdb/train/unsup/44959_0.txt
aclImdb/train/unsup/44958_0.txt
aclImdb/train/unsup/44957_0.txt
aclImdb/train/unsup/44956_0.txt
aclImdb/train/unsup/44955_0.txt
aclImdb/train/unsup/4

In [ ]:
!pip3 install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=54aa48a9d5b2279e12565339b19362dd83077e7ea55bf46547ffd9f43a1c7da7
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
# import libraries
import os
import random
import numpy as np
import pandas as pd
import scipy
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tag import pos_tag
import re
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt
import itertools
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
import gensim
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# Define a function to load the data
def dataloader(data_path):
    data = {}
    # Iterate through "train" and "test" folders
    for folder in ["train", "test"]:
        # Initialize an empty list for each folder to store data
        data[folder] = []
        # Iterate through sentiment classes: "neg" (negative) and "pos" (positive)
        for sent_class in ["neg", "pos"]:
            # Assign label 1 for positive sentiment and 0 for negative sentiment
            if sent_class == "pos":
                lable = 1
            else:
                lable = 0
            # Create path to the current sentiment class folder
            path = os.path.join(data_path, folder, sent_class)
            # Get a list of files in the current folder
            files = os.listdir(path)
            # Open the file and read its contents
            for file in files:
                with open(os.path.join(path, file), "r") as f:
                    sample = f.read()
                    # Append the sample and its label to the data list
                    data[folder].append([sample, lable])
    # Set the random seed for reproducibility
    random.seed(5)
     # Shuffle the training and testing data
    random.shuffle(data["train"])
    random.shuffle(data["test"])
    # Convert the lists of samples and labels to Pandas DataFrames
    data["train"] = pd.DataFrame(data["train"], columns=['text', 'sentiment'])
    data["test"]  = pd.DataFrame(data["test"] , columns=['text', 'sentiment'])
    # Return the training and testing DataFrames
    return data["train"], data["test"]

In [ ]:
# get the dataframes
train_data, test_data = dataloader('/content/aclImdb')

In [ ]:
# save them for later use
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

In [ ]:
# check the first 5 rows
train_data.head()

,text,sentiment,clean_text,clean_text_normalized,clean_text_w2v_no_stop,clean_text_bert
0,"I confess--Emma, in my opinion, is the single ...",0,confess emma in my opinion is the single great...,confess emma in my opinion be the singl greate...,confess emma opinion single greatest novel eve...,"i confess--emma, in my opinion, is the single ..."
1,I have given this film an elevated rating of 2...,0,have given this film an elevated rating of sta...,have give thi film an elev rate of star as per...,given film elevated rating stars personally ap...,i have given this film an elevated rating of 2...
2,"I've watched this movie twice now on DVD, and ...",1,have watched this movie twice now on dvd and b...,have watch thi movi twice now on dvd and both ...,watched movie twice dvd times not fail impress...,"i have watched this movie twice now on dvd, an..."
3,Star Trek Hidden Frontier will surprise you in...,1,star trek hidden frontier will surprise you in...,star trek hide frontier will surpris you in ma...,star trek hidden frontier surprise many ways f...,star trek hidden frontier will surprise you in...
4,A very early Oliver Stone (associate-)produced...,1,very early oliver stone associate produced fil...,veri earli oliv stone associ produc film and o...,early oliver stone associate produced film one...,a very early oliver stone (associate-)produced...


# **Data cleaning**

In [ ]:
def preprocessing_POS(text):
    # Tokenize the input text into a list of words
    word_list = word_tokenize(text)
    # Initialize an empty list to store the processed words
    temp = []
    # Create instances of WordNetLemmatizer and PorterStemmer
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    # Iterate through each word and its POS tag in the tokenized text
    for word, tag in pos_tag(word_list):
        # Lemmatize based on the POS tag
        if tag.startswith('J'):
            wrd = lemmatizer.lemmatize(word, pos='a') # Adjective
        elif tag.startswith('V'):
            wrd = lemmatizer.lemmatize(word, pos='v') # Verb
        elif tag.startswith('N'):
            wrd = lemmatizer.lemmatize(word, pos='n') # Noun
        elif tag.startswith('R'):
            wrd = lemmatizer.lemmatize(word, pos='r') # Adverb
        else:
            # Keep the word unchanged if not one of the specified POS tags
            wrd = word
        # Apply stemming to the lemmatized word
        wrd = stemmer.stem(wrd)
        # Append the processed word to the list
        temp.append(wrd)
    # Join the processed words into a string, separated by spaces
    sample = ' '.join(temp)
    # Return the final processed review
    return sample

In [ ]:
def cleanText(text):

    # Remove HTML tags
    text = re.sub(r'<.*?>', ' ', text)

    # Expand contractions
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"'ve", " have", text)
    text = re.sub(r"'ll", " will", text)
    text = re.sub(r"'re", " are", text)

    # Conditional cleaning based on embedding type
    if embedding != 'BERT':
        # Remove digits and hyphens
        text = re.sub(r"[0-9]+", ' ', text)
        text = re.sub(r"-", ' ', text)

    # Convert to lowercase and strip whitespace
    text = text.strip().lower()

    # Additional cleaning for specific embeddings
    if embedding == 'WORD2VEC_NO_STOP':
        # Remove stopwords
        default_stop_words = set(stopwords.words('english'))
        default_stop_words.difference_update({'no', 'not', 'nor', 'too', 'any'})
        stop_words = default_stop_words.union({"'m", "n't", "'d", "'re", "'s", 'would','must',"'ve","'ll",'may'})
        word_list = word_tokenize(text)
        filtered_list = [w for w in word_list if not w in stop_words]
        text = ' '.join(filtered_list)

    if embedding != 'BERT':
        # Remove other contractions
        text = re.sub(r"'", ' ', text)

    # Replace punctuations with space
    if embedding == 'BERT':
        # Preserve certain punctuation marks for sentence detection in BERT
        filters = '"#$%&*+<=>@[\]^_`{|}~\t\n'
        text = re.sub(r'\!+', '!', text)
        text = re.sub(r'\?+', '?', text)
    else:
        # Define standard filters for other embeddings
        filters = '!"\'#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n'

    # Use translation to remove punctuations
    translate_dict = dict((i, " ") for i in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)

    if embedding == 'BERT':
        # Replace '*' with space for BERT
        text = re.sub(r'\*', ' ', text)

    if embedding != 'BERT':
        # Remove single-character words for non-BERT embeddings
        text = ' '.join([w for w in text.split() if len(w) > 1])

    # Replace multiple spaces with a single space
    text = re.sub(' +', ' ', text)
    text = ''.join(text)

    return text

In [ ]:
embedding = ''
# for Word2Vec with stop words
train_data['clean_text'] = train_data['text'].apply(cleanText)
test_data['clean_text'] = test_data['text'].apply(cleanText)

In [ ]:
# for BOW and TFIDF(s)
train_data['clean_text_normalized'] = train_data['clean_text'].apply(preprocessing_POS)
test_data['clean_text_normalized'] = test_data['clean_text'].apply(preprocessing_POS)

In [ ]:
# Filter out words that occur less than 5 times in the training data
re_train = pd.Series(' '.join(train_data['clean_text_normalized']).split()).value_counts()
re_train = re_train[(re_train <5)]

In [ ]:
# Filter out words that occur less than 5 times in the test data
re_test = pd.Series(' '.join(test_data['clean_text_normalized']).split()).value_counts()
re_test = re_test[(re_test <5)]

In [ ]:
# Remove these words
train_data['clean_text_normalized'] = train_data['clean_text_normalized'].apply(lambda x: ' '.join(x for x in x.split() if x not in re_train))
test_data['clean_text_normalized'] = test_data['clean_text_normalized'].apply(lambda x: ' '.join(x for x in x.split() if x not in re_test))


In [ ]:
embedding = 'WORD2VEC_NO_STOP'
# for Word2Vec without stop words
train_data['clean_text_w2v_no_stop'] = train_data['text'].apply(cleanText)
test_data['clean_text_w2v_no_stop'] = test_data['text'].apply(cleanText)

In [ ]:
embedding = 'BERT'
# for BERT(s)
train_data['clean_text_bert'] = train_data['text'].apply(cleanText)
test_data['clean_text_bert'] = test_data['text'].apply(cleanText)

In [ ]:
# print the results
print("A review example of dataset before cleaning:", end="\n\n")
print(train_data.iloc[3]['text'], end='\n\n')

print("clean_text:")
print(train_data.iloc[3]['clean_text'], end="\n\n")

print("clean_text_normalized:")
print(train_data.iloc[3]['clean_text_normalized'], end="\n\n")

print("clean_text_w2v_no_stop:")
print(train_data.iloc[3]['clean_text_w2v_no_stop'], end="\n\n")

print("clean_text_bert:")
print(train_data.iloc[3]['clean_text_bert'], end="\n\n")

A review example of dataset before cleaning:

Star Trek Hidden Frontier will surprise you in many ways. First, it's a fan made series, available only on the web, and it features mainly friends & neighbors who have the computer programs and home video cameras and sewing machines to, as Mickey & Judy once put it, put on a show. It's definitely friends & neighbors to, you can tell. A lot of these people aren't the most beautiful looking folks you've ever seen, or the youngest, or the thinnest some of them stumble through their lines like they're walking on marbles some of them have thick accents, or simply don't seem to speak well in the first place, whick makes it virtually impossible to understand a single solitary word that they're saying. Still, you have to admit, for everything these friends & neighbors have put together, it's actually fun to watch. Yes, some of the dialogue is hokey. Yes, it's a little odd (though admittedly a little cool too) watching two Starfleet males kiss (al

In [ ]:
!cp /content/test_data.csv /content/drive/MyDrive/NLP_Proj

In [ ]:
!cp /content/train_data.csv /content/drive/MyDrive/NLP_Proj

# **Vectorization**

In [ ]:
def mean(z):
    return sum(itertools.chain(z))/len(z)

**BOW**

In [ ]:
# Specify the type of embedding method to be used
embedding = 'BOW'

# Create a CountVectorizer object with English stop words
vectorizer = CountVectorizer(stop_words='english')

# Apply the CountVectorizer on the cleaned and normalized text data for training and testing
bow_training_features = vectorizer.fit_transform(train_data["clean_text_normalized"])
bow_test_features = vectorizer.transform(test_data["clean_text_normalized"])

# Print the shape of the resulting Bag of Words (BOW) matrices for training and testing data
print("Shape of BOW training features:", bow_training_features.shape)
print("Shape of BOW test features:", bow_test_features.shape)

(25000, 19368)
(25000, 19368)


In [ ]:
scipy.sparse.save_npz("bow_training_features.npz", bow_training_features)

In [ ]:
scipy.sparse.save_npz("bow_test_features.npz", bow_test_features)

In [ ]:
!cp bow_training_features.npz /content/drive/MyDrive/NLP_Proj
!cp /content/bow_test_features.npz /content/drive/MyDrive/NLP_Proj

In [ ]:
# Specify the type of embedding method to be used
embedding = 'TFIDF_WITH_STOP'

# Create a TfidfVectorizer object with specified parameters
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=20000)

# Apply the TfidfVectorizer on the cleaned and normalized text data for training and testing
tfidf_with_stop_training_features = vectorizer.fit_transform(train_data["clean_text_normalized"])
tfidf_with_stop_test_features = vectorizer.transform(test_data["clean_text_normalized"])

# Print the shape of the resulting TF-IDF matrices for training data
print("Shape of TF-IDF with stop words training features:", tfidf_with_stop_training_features.shape)

(25000, 20000)


In [ ]:
scipy.sparse.save_npz("tfidf_with_stop_training_features.npz", tfidf_with_stop_training_features)
scipy.sparse.save_npz("tfidf_with_stop_test_features.npz", tfidf_with_stop_test_features)

In [ ]:
!cp tfidf_with_stop_training_features.npz /content/drive/MyDrive/NLP_Proj
!cp tfidf_with_stop_test_features.npz /content/drive/MyDrive/NLP_Proj

In [ ]:
# Specify the type of embedding method to be used, in this case
embedding = 'TFIDF_NO_STOP'

# Create a TfidfVectorizer object with specified parameters, including removing English stop words
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_features=20000)

# Apply the TfidfVectorizer on the cleaned and normalized text data for training and testing
tfidf_no_stop_training_features = vectorizer.fit_transform(train_data["clean_text_normalized"])
tfidf_no_stop_test_features = vectorizer.transform(test_data["clean_text_normalized"])

# Print the shape of the resulting TF-IDF matrices for training and testing data
print("Shape of TF-IDF without stop words training features:", tfidf_no_stop_training_features.shape)
print("Shape of TF-IDF without stop words test features:", tfidf_no_stop_test_features.shape)

(25000, 20000)
(25000, 20000)


In [ ]:
scipy.sparse.save_npz("tfidf_no_stop_training_features.npz", tfidf_no_stop_training_features)
scipy.sparse.save_npz("tfidf_no_stop_test_features.npz", tfidf_no_stop_test_features)

In [ ]:
!cp tfidf_no_stop_training_features.npz /content/drive/MyDrive/NLP_Proj
!cp tfidf_no_stop_test_features.npz /content/drive/MyDrive/NLP_Proj

**Word2Vec**

In [ ]:
def embeddToWord2Vec(text):
    # Tokenize the input text into a list of words
    words = word_tokenize(text)

    # Check the specified embedding type
    if embedding == 'WORD2VEC_WITH_STOP':
        # Use the Word2Vec model with stop words
        result = [w2v_with_stop_model.wv[w] for w in words if w in w2v_with_stop_model.wv.key_to_index]
    else:
        # Use the Word2Vec model without stop words
        result = [w2v_no_stop_model.wv[w] for w in words if w in w2v_no_stop_model.wv.key_to_index]

    # Calculate the mean vector for the word embeddings
    feature = [mean(x) for x in zip(*result)]

    return feature

In [ ]:
def wordTokenize(text):
    # Tokenize the input text into a list of words using NLTK's word_tokenize function
    return word_tokenize(text)

In [ ]:
embedding = 'WORD2VEC_WITH_STOP'

In [ ]:
# Tokenize the cleaned text in the 'train_data' DataFrame using the 'wordTokenize' function
words = train_data['clean_text'].apply(wordTokenize)

# Train a Word2Vec model using the tokenized words
w2v_with_stop_model = gensim.models.Word2Vec(words, min_count=2, vector_size=100, window=5)

In [ ]:
# Apply the 'embeddToWord2Vec' function to the cleaned text in the 'train_data' DataFrame
word2vec_with_stop_training_features = train_data['clean_text'].apply(embeddToWord2Vec)

# Apply the 'embeddToWord2Vec' function to the cleaned text in the 'test_data' DataFrame
word2vec_with_stop_test_features = test_data['clean_text'].apply(embeddToWord2Vec)

In [ ]:
# Transpose the Word2Vec features for the training set
feature = [x for x in word2vec_with_stop_training_features.transpose()]
word2vec_with_stop_training_features = np.asarray(feature)

# Transpose the Word2Vec features for the testing set
feature = [x for x in word2vec_with_stop_test_features.transpose()]
word2vec_with_stop_test_features = np.asarray(feature)

# Print the shape of the resulting transposed Word2Vec feature matrices
print(word2vec_with_stop_training_features.shape)
print(word2vec_with_stop_test_features.shape)

(25000, 100)
(25000, 100)


In [ ]:
np.save('word2vec_with_stop_training_features.npy', word2vec_with_stop_training_features)
np.save('word2vec_with_stop_test_features.npy', word2vec_with_stop_test_features)

In [ ]:
!cp word2vec_with_stop_training_features.npy /content/drive/MyDrive/NLP_Proj
!cp word2vec_with_stop_test_features.npy /content/drive/MyDrive/NLP_Proj

In [ ]:
embedding = 'WORD2VEC_NO_STOP'

In [ ]:
# Tokenize the cleaned text (Word2Vec without stop words) in the 'train_data' DataFrame
words = train_data['clean_text_w2v_no_stop'].apply(wordTokenize)

# Train a Word2Vec model without stop words using the tokenized words
# The parameters include: minimum word count, vector size, and context window size
w2v_no_stop_model = gensim.models.Word2Vec(words, min_count=2, vector_size=100, window=5)


In [ ]:
# Apply the 'embeddToWord2Vec' function to the cleaned text without stop words in the 'train_data' DataFrame
word2vec_no_stop_training_features = train_data['clean_text_w2v_no_stop'].apply(embeddToWord2Vec)

# Apply the 'embeddToWord2Vec' function to the cleaned text without stop words in the 'test_data' DataFrame
word2vec_no_stop_test_features = test_data['clean_text_w2v_no_stop'].apply(embeddToWord2Vec)

In [ ]:
# Transpose the Word2Vec features for the training set without stop words
feature = [x for x in word2vec_no_stop_training_features.transpose()]
word2vec_no_stop_training_features = np.asarray(feature)

# Transpose the Word2Vec features for the testing set without stop words
feature = [x for x in word2vec_no_stop_test_features.transpose()]
word2vec_no_stop_test_features = np.asarray(feature)

# Print the shape of the resulting transposed Word2Vec feature matrices without stop words
print(word2vec_no_stop_training_features.shape)
print(word2vec_no_stop_test_features.shape)

(25000, 100)
(25000, 100)


In [ ]:
np.save('word2vec_no_stop_training_features.npy', word2vec_with_stop_training_features)
np.save('word2vec_no_stop_test_features.npy', word2vec_with_stop_test_features)

In [ ]:
!cp word2vec_no_stop_training_features.npy /content/drive/MyDrive/NLP_Proj
!cp word2vec_no_stop_test_features.npy /content/drive/MyDrive/NLP_Proj

**BERT**

In [ ]:
# Cleaning before BERT
embedding = 'BERT'

In [ ]:
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
from numpy import mean

In [ ]:
! pip install git+https://github.com/khalidsaifullaah/BERTify

  Cloning https://github.com/khalidsaifullaah/BERTify to /tmp/pip-req-build-tqtzoabw
  Running command git clone --filter=blob:none --quiet https://github.com/khalidsaifullaah/BERTify /tmp/pip-req-build-tqtzoabw
  Resolved https://github.com/khalidsaifullaah/BERTify to commit c7cefb9517462ff1d123c216dd3d6ffe377e2ef0
  Preparing metadata (setup.py) ... done
  Created wheel for BERTify: filename=BERTify-0.0.3-py3-none-any.whl size=6229 sha256=7d5b341ea426cc7708921e5bf4ec2d640590b90077f472d0269735af8fcaddfd
  Stored in directory: /tmp/pip-ephem-wheel-cache-oqqce1g6/wheels/e5/7a/6d/c1beca78664e7bdf4e88c959ee84e5cb4db5531cab6570c278
Successfully built BERTify


In [ ]:
from bertify import BERTify

In [ ]:
# Create a BERTify object for English language
en_bertify = BERTify(
    lang="en",
    last_four_layers_embedding=False
)

# Set the batch size for BERTify to 96
en_bertify.batch_size = 96


##########
Using cuda for computation
using bert-base-uncased model for en embedding
##########


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# Obtain BERT embeddings for the cleaned text in the 'clean_text_bert' column of the training data
bert_word_training_features = en_bertify.embedding(train_data['clean_text_bert'])

  0%|          | 0/261 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [ ]:
np.save('bert_word_training_features.npy', bert_word_training_features)

In [ ]:
# Obtain BERT embeddings for the cleaned text in the 'clean_text_bert' column of the test data
bert_word_test_features = en_bertify.embedding(test_data['clean_text_bert'])

  0%|          | 0/261 [00:00<?, ?it/s]

In [ ]:
np.save('bert_word_test_features.npy', bert_word_test_features)

In [ ]:
print(bert_word_training_features.shape)
print(bert_word_test_features.shape)

(25000, 768)
(25000, 768)


In [ ]:
!cp bert_word_training_features.npy /content/drive/MyDrive/NLP_Proj
!cp bert_word_test_features.npy /content/drive/MyDrive/NLP_Proj

In [ ]:
# Create a SentenceTransformer model using the 'bert-base-nli-mean-tokens' pre-trained model
bert_transformers = SentenceTransformer('bert-base-nli-mean-tokens')

.gitattributes:   0%|          | 0.00/391 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
# Define a function to embed text using BERT embeddings
def embeddToBERT(text):
    # Split the text into sentences based on common sentence-ending punctuation (!, ?, .)
    sentences = re.split('!|\?|\.', text)

    # Remove any empty sentences after splitting
    sentences = list(filter(None, sentences))

    # Encode the sentences using the SentenceTransformer model (BERT-based)
    result = bert_transformers.encode(sentences)

    # Calculate the mean vector for the sentence embeddings
    feature = [mean(x) for x in zip(*result)]

    # Return the resulting feature vector representing the BERT embeddings for the text
    return feature

In [ ]:
# Apply the 'embeddToBERT' function to the cleaned text in the 'clean_text_bert' column of the training data
bert_sentence_training_features = train_data['clean_text_bert'].apply(embeddToBERT)

# Apply the 'embeddToBERT' function to the cleaned text in the 'clean_text_bert' column of the testing data
bert_sentence_test_features = test_data['clean_text_bert'].apply(embeddToBERT)

In [ ]:
# Transpose the BERT sentence features for the training set
feature = [x for x in bert_sentence_training_features.transpose()]
bert_sentence_training_features = np.asarray(feature)

# Transpose the BERT sentence features for the testing set
feature = [x for x in bert_sentence_test_features.transpose()]
bert_sentence_test_features = np.asarray(feature)

# Print the shape of the resulting transposed BERT sentence feature matrices
print(bert_sentence_training_features.shape)
print(bert_sentence_test_features.shape)

(25000, 768)
(25000, 768)


In [ ]:
np.save('bert_sentence_training_features.npy', bert_sentence_training_features)
np.save('bert_sentence_test_features.npy', bert_sentence_test_features)

In [ ]:
!cp bert_sentence_training_features.npy /content/drive/MyDrive/NLP_Proj
!cp bert_sentence_test_features.npy /content/drive/MyDrive/NLP_Proj